In [4]:
# auto reload the module
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from dataclasses import dataclass
from datasets import load_dataset
import pickle
import matplotlib.pyplot as plt
import numpy as np
import torch 
from torch.utils.data import TensorDataset
from diffusers import UNet2DModel, UNet2DConditionModel
from PIL import Image
from diffusers import DDPMScheduler
import torch.nn.functional as F
from diffusers.optimization import get_cosine_schedule_with_warmup
from diffusers import DDPMPipeline
import math
import os
from tqdm.auto import tqdm
from torchvision import transforms
from flow_generator import FlowGenerator

torch.manual_seed(0)

In [6]:
@dataclass
class TrainingConfig:
    '''
    Class for training parameters
    '''
    
    image_size = 128  # the generated image resolution
    train_batch_size = 4
    eval_batch_size = 4  # how many images to sample during evaluation
    num_epochs = 200
    gradient_accumulation_steps = 1
    learning_rate = 1e-4
    lr_warmup_steps = 500
    save_image_epochs = 10
    save_model_epochs = 30
    mixed_precision = "fp16"  # `no` for float32, `fp16` for automatic mixed precision
    output_dir = "/home/maddie/Documents/underwater/DeepCFD/output-conditional"  # the model name locally and on the HF Hub

    push_to_hub = False  # whether to upload the saved model to the HF Hub
    hub_private_repo = False
    overwrite_output_dir = True  # overwrite the old model when re-running the notebook
    seed = 0


config = TrainingConfig()

In [7]:
# Load the pickle files using the exact file paths
c_data_path = "/home/maddie/Documents/underwater/DeepCFD/dataX.pkl"
y_data_path = "/home/maddie/Documents/underwater/DeepCFD/dataY.pkl"

# conditional data 
with open(c_data_path, "rb") as f:
    c = pickle.load(f)

# output data (data we want to predict )
with open(y_data_path, "rb") as f:
    y = pickle.load(f)

In [8]:
# Preprocess the data

# turn the input in a pytorch tensor
c = torch.FloatTensor(c)
y = torch.FloatTensor(y)

# Normalize the data [-1,1] for each channel 
def normalize_tensor(tensor):
    min_val = torch.amin(tensor, dim=(0, 2, 3), keepdim=True)  # min over all axes except channels
    max_val = torch.amax(tensor, dim=(0, 2, 3), keepdim=True)  # max over all axes except channels
    # Normalize to [-1, 1]
    tensor_normalized = 2 * ((tensor - min_val) / (max_val - min_val)) - 1
    return tensor_normalized

# normalize the input and output data
c = normalize_tensor(c)
y = normalize_tensor(y)


In [9]:

# Function for dividing the dataset 
def split_tensors(*tensors, ratio):
    preprocess = transforms.Compose(
        [
            transforms.Resize((128,128)),
            transforms.RandomHorizontalFlip(),
            # transforms.ToTensor(),
            # transforms.Normalize([0.5], [0.5]),
        ]
    )

    assert len(tensors) > 0
    split1, split2 = [], []
    count = len(tensors[0])
    for tensor in tensors:
        assert len(tensor) == count
        tensor = [preprocess(temp) for temp in tensor]
        tensor = torch.stack(tensor)
        split1.append(tensor[:int(len(tensor) * ratio)])
        split2.append(tensor[int(len(tensor) * ratio):])
    if len(tensors) == 1:
        split1, split2 = split1[0], split2[0]
    return split1, split2

# Split the data into training and testing sets (70/30)
train_data, test_data = split_tensors(c,y,ratio=0.7)

# train_data and test_data are lists containing two tensors each: [inputs, outputs]
train_dataset = TensorDataset(*train_data)
test_dataset = TensorDataset(*test_data)

# Print the shapes of the tensors in train_data
train_data_c = train_data[0]
train_data_y = train_data[1]

# Print the shapes of the tensors in test_data
test_data_c = test_data[0]
test_data_y = test_data[1]

# Load data into PyTorch DataLoader
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=config.train_batch_size, shuffle=True)

test_c, test_y = test_data[:] # split test data into x and y



In [10]:
# Create Scheduler 
noise_scheduler = DDPMScheduler(num_train_timesteps=10000) # DDPM scheduler with 1000 timesteps

In [11]:
model = FlowGenerator(
    in_channels = 3,
    out_channels = 3, 
    geometry_channels = 3,
    layers_per_block=2,
    down_block_types =(
        "DownBlock2D",  # a regular ResNet downsampling block
        "DownBlock2D",
        "DownBlock2D",
        "DownBlock2D",
        "AttnDownBlock2D",  # a ResNet downsampling block with spatial self-attention
        "DownBlock2D"
    ),
    mid_block_type = "UNetMidBlock2DCrossAttn",
    up_block_types = (
        "UpBlock2D",  # a regular ResNet upsampling block
        "AttnUpBlock2D",  # a ResNet upsampling block with spatial self-attention
        "UpBlock2D",
        "UpBlock2D",
        "UpBlock2D",
        "UpBlock2D",
    ),
    geometry_block_types = (
        "DownBlock2D",  # a regular ResNet downsampling block
        "DownBlock2D",
        "DownBlock2D",
        "DownBlock2D",
        "AttnDownBlock2D",  # a ResNet downsampling block with spatial self-attention
        "DownBlock2D"
    ),
    block_out_channels = (128, 128, 256, 256, 512, 512),
)


In [12]:
weight_path = "/home/maddie/Documents/underwater/DeepCFD/output-conditional/model_epoch_149.pth"

# load weight to model
model.load_state_dict(torch.load(weight_path))
model.cuda()
_ = model.eval()

In [21]:
_, _, H, W = y.shape

postprocess = transforms.Compose(
        [
            transforms.Resize((H,W)),
            # transforms.RandomHorizontalFlip(),
            # transforms.ToTensor(),
            # transforms.Normalize([0.5], [0.5]),
        ]
    )

def plot_condition_prediction_truth(conditions, predictions, ground_truth):
    # Assuming conditions, predictions, and ground_truth are PyTorch tensors of shape [N, C, H, W]

    fig, axs = plt.subplots(3, 3, figsize=(15,5))
    
    for i in range(3):
        # Display condition
        ax = axs[i, 0]
        condition_img = conditions[i].cpu().numpy().transpose(1, 2, 0)
        ax.imshow((condition_img * 0.5 + 0.5))  # Assuming normalization was [-1, 1]
        ax.set_title("Condition")
        ax.axis('off')
        
        # Display prediction
        ax = axs[i, 1]
        prediction_img = predictions[i].cpu().numpy().transpose(1, 2, 0)
        ax.imshow((prediction_img * 0.5 + 0.5))  # Assuming normalization was [-1, 1]
        ax.set_title("Prediction")
        ax.axis('off')
        
        # Display ground truth if provided
        ax = axs[i, 2]
        ground_truth_img = ground_truth[i].cpu().numpy().transpose(1, 2, 0)
        ax.imshow((ground_truth_img * 0.5 + 0.5))  # Assuming normalization was [-1, 1]
        ax.set_title("Ground Truth")
        ax.axis('off')

    plt.tight_layout()
    plt.show()

In [22]:
noise_scheduler.timesteps

tensor([9999, 9998, 9997,  ...,    2,    1,    0])

In [23]:
with torch.no_grad():
    # Assuming test_conditions and test_y are your test set conditions and ground truths
    input_noise = input = torch.randn((1, 3, 128, 128)).to("cuda")
    test_conditions = next(iter(test_c)).to("cuda")  # assuming condition is the first item in the dataset tuple
    predictions = []
    for t in tqdm(noise_scheduler.timesteps):
        noisy_residual = model(sample = input_noise, 
                               condition = test_conditions, 
                               time = t)
        previous_noisy_sample = noise_scheduler.step(noisy_residual, t, input_noise)
        input_noise = previous_noisy_sample.prev_sample

    predictions = input_noise  # Final predictions after the loop

# Assuming normalization was done as [-1,1], adjust if different
plot_condition_prediction_truth(test_conditions, predictions, test_y)

  0%|          | 0/10000 [00:00<?, ?it/s]

RuntimeError: Expected weight to be a vector of size equal to the number of channels in input, but got weight of shape [128] and input of shape [128, 64, 64]